Check ComCam data for known Strong-lens systems

Author: Michael Wood-Vasey <wmwv@pitt.edu>

Last run: 2024 November 20

Check the ComCam-observed regions of the Chandra Deep Field South for locations of known strong lenses from the Extended Chandra Deep Field South program with HST/GEMS survey:   
https://iopscience.iop.org/0004-637X/734/1/69/suppdata/apj390026t1_ascii.txt?doi=10.1088/0004-637X/734/1/69

In [ ]:
table1 = \
"""
Table 1 
GEMS Strong Lens Candidates


ID^a	Tile^b	R.A.^c	Decl.^c	V_tot^d	z_tot^d	z_l^e	z_lsp^f	Component^g	log(M_t*)^h	log(M_*)^i	b'^j	q^k	log(M_E)^l	
15422	44	03:32:38.21	-27:56:53.2	 sdotsdotsdot 	19.20	0.58	0.62	1+3	11.4	11.22^+0.22 _- 0.20	1.20	0.40	11.80^+0.15 _- 0.06	
34244	94	03:32:06.45	-27:47:28.6	23.43	21.11	1.00	1.02	1+4	11.0	10.82^+0.20 _- 0.29	0.40	0.48	11.04^+0.27 _- 0.11	
40173	35	03:33:19.45	-27:44:50.0	20.86	19.67	0.44	0.42	2+3	10.7	10.58^+0.13 _- 0.14	0.59	1.00	10.85^+0.09 _- 0.04	
43242	45	03:31:55.35	-27:43:23.5	21.85	20.00	0.66	0.66	2+3	11.1	11.04^+0.18 _- 0.14	0.57	0.60	11.08^+0.16 _- 0.07	
46446	47	03:31:35.94	-27:41:48.2	23.60	21.70	0.88	 sdotsdotsdot 	2+3	10.6	10.37^+0.23 _- 0.21	0.42*	1.00	10.90^+0.23 _- 0.09	
Notes.
a ID from the COMBO-17 catalog.
b GEMS tile number (1-63 GEMS, 80-95 GOODS region).
c Positions of candidates are in J2000 with units of hr:minutes:s and deg:minutes:s.
d Apparent V = F606W and z = F814LP HST/ACS magnitudes of the whole system (Caldwell et al. 2008).
e Photometric redshift of lensing galaxy from COMBO-17.
f Spectroscopic redshift of lensing galaxy (see Section 5.2 for details).
g Number of components used in GALFIT composition for lensing galaxy + lensed images.
h Total stellar mass of the lens galaxy in units of M_sun from Borch et al. (2006).
i Stellar mass of the lens galaxy inside the critical radius in the units of M_sun.
j Einstein radius in arcseconds corresponding to SIE model except * corresponds to SIS+shear model.
k Axis ratio from the lens mass model.
l Mass within the critical curve in units of M_sun. The upper and lower limits correspond to the 16th and 84th percentile of the lens mass PDF.
"""

In [ ]:
import matplotlib.pyplot as plt

from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay

import lsst.geom

import lsst.afw.image
from lsst.afw.math import Warper, WarperConfig
import lsst.afw.table

from lsst.ip.diffim import AlardLuptonSubtractConfig, AlardLuptonSubtractTask
from lsst.ip.diffim import GetTemplateConfig, GetTemplateTask
from lsst.ip.diffim import DetectAndMeasureConfig, DetectAndMeasureTask

from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
afwDisplay.setDefaultBackend("firefly")

In [ ]:
data = Table(rows=[
    (15422, 44, "03:32:38.21", "-27:56:53.2"),
    (34244, 94, "03:32:06.45", "-27:47:28.6"),
    (40173, 35, "03:33:19.45", "-27:44:50.0"),
    (43242, 45, "03:31:55.35", "-27:43:23.5"),
    (46446, 47, "03:31:35.94", "-27:41:48.2")],
    names=("id", "tile", "ra", "dec"),
    dtype=(int, int, str, str),
)

In [ ]:
data["skycoor"] = SkyCoord(data["ra"], data["dec"], unit=(u.hourangle, u.degree))

In [ ]:
# Load collection
repo = "embargo_new"
# collection = "LSSTComCam/nightlyValidation"
# As of 2024-11-12 there are observations from 2024-11-08, 2024-11-09, 2024-11-10, 2024-11-11
# The idea was to get the nightly coadd from the template and the science night
# but the nightly coadd for 2024-11-11 doesn't cover the region of the first lens system
# although there are some individual exalepxs that do.
collection_template = "LSSTComCam/runs/nightlyValidation/20241108/d_2024_11_05/DM-47059"
collection_science = "LSSTComCam/runs/nightlyValidation/20241111/d_2024_11_05/DM-47059"

instrument = "LSSTComCam"

butler_template = Butler(repo, collections=collection_template)
butler_science = Butler(repo, collections=collection_science)

name_skymap = "lsst_cells_v1"
skymap = butler_template.get("skyMap", skymap=name_skymap, collections="skymaps")

In [ ]:
### Find a single image from the night

In [ ]:
i = 0
ra, dec = data["skycoor"][i].ra.value, data["skycoor"][i].dec.value

datasetRefs_science = butler_science.query_datasets("calexp", where="visit_detector_region.region OVERLAPS POINT(ra, dec) and band='i'",
                                    bind={"ra": ra, "dec": dec})

In [ ]:
for dr in datasetRefs_science:
    print(dr)

In [ ]:
i = 2
calexp = butler_science.get("calexp", dataId=datasetRefs_science[i].dataId)
x, y = calexp.getWcs().skyToPixel(lsst.geom.SpherePoint(ra, dec, units=lsst.geom.degrees))

In [ ]:
print(x, y)

In [ ]:
datasetRefs_science[0].dataId

In [ ]:
afw_display = afwDisplay.Display(frame=1)

def cutout(im, x, y, size=100):
    return im.getCutout(lsst.geom.Point2D(x, y), lsst.geom.Extent2I(size, size))

afw_display.scale("asinh", -2, 5)
afw_display.mtv(cutout(calexp, x, y))
afw_display.setMaskTransparency(100)

### Find nightly coadd
Get tract, patch from ra, dec

In [ ]:
coord = lsst.geom.SpherePoint(ra, dec, units=lsst.geom.degrees)
skymap.findTractPatchList([coord])

I wrote down the following tract, patch from the previous output.  There's certainly a programatic way to do this that I forget right now.

In [ ]:
tract, patch = (5063, 24)
band = "i"

In [ ]:
template_coadd = butler_science.get(
            "deepCoadd_calexp",
            band=band, patch=patch, tract=tract, skymap=name_skymap
        )

coadd_x, coadd_y = template_coadd.getWcs().skyToPixel(lsst.geom.SpherePoint(ra, dec, units=lsst.geom.degrees))
print(coadd_x, coadd_y)

In [ ]:
afw_display = afwDisplay.Display(frame=2)

afw_display.scale("asinh", -1, 3)
# afw_display.scale("linear", "zscale")
afw_display.setMaskTransparency(100)
afw_display.mtv(cutout(template_coadd, coadd_x, coadd_y))
print(collection_template)